In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Libraries

In [ ]:
import pandas as pd 

## Load Data

In [ ]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.head()

### Initial Data Exploration
* Display the first few rows of the dataset.
* Display basic information about the dataset.
* Display summary statistics of the dataset.

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

### Handle Missing Values
Check for missing values in the dataset.
Fill missing values appropriately.

In [ ]:
df.drop("Cabin", axis =1 , inplace=True)

In [ ]:
df

In [ ]:
mean_age = df['Age'].mean()

In [ ]:
df['Age'].fillna(mean_age, inplace=True)
df.isnull().sum()

In [ ]:
df['Embarked'].fillna(method="ffill", inplace=True)
df.isnull().sum()

### Convert Categorical Variables
Convert categorical variables to numerical format using appropriate techniques.

In [ ]:
dummies = pd.get_dummies(df['Embarked'], dtype='int', drop_first=True)

In [ ]:
df = pd.concat([df, dummies], axis = 1)
df.head()
df.drop('Embarked', axis = 1, inplace=True)
df.head()

In [ ]:
df.Pclass.dtype

In [ ]:
df['Pclass'] = df['Pclass'].astype(str) #converts column type to str
dummies=pd.get_dummies(df[['Sex','Pclass']], drop_first=True, dtype='int')

In [ ]:
df = pd.concat([df, dummies], axis = 1)
df.head()

In [ ]:
df.drop(["PassengerId","Name", "Pclass", "Sex", "Ticket", "Fare"], axis =1, inplace=True)
df.head()

In [ ]:
def pre_process(data):
    data['Age'].fillna(mean_age, inplace=True)
    data['Embarked'].fillna(method = "ffill", inplace=True)
    data['Pclass'] = data['Pclass'].astype(str)
    dummies = pd.get_dummies(data[['Sex', 'Pclass','Embarked']], \
                            drop_first=True, dtype='int')
    data = pd.concat([data, dummies], axis = 1)
    data.drop(["PassengerId","Sex", "Pclass", "Cabin","Ticket", \
               "Fare", "Embarked","Name"], axis =1, inplace=True)
    return data 

In [ ]:
df_new = pd.read_csv('/kaggle/input/titanic/train.csv')
y = df_new['Survived']
X = df_new.drop("Survived",axis = 1)
X = pre_process(X)
X.head()

In [ ]:
train_len = int(len(X)*0.7)
valid_len = len(X) - train_len
# print(train_len, valid_len)

X_train = X.iloc[0:train_len,:]
X_valid = X.iloc[train_len:, :]
y_train = y[0:train_len]
y_valid = y[train_len:]
print(len(X_train), len(X_valid),len(y_train), len(y_valid))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_valid, y_valid)

In [ ]:
df_test =pd.read_csv('/kaggle/input/titanic/test.csv')
pass_id = df_test['PassengerId']
df_test.head()

In [ ]:
df_test = pre_process(df_test)
df_test.head()

In [ ]:
predictions = model.predict(df_test)

In [ ]:
submission = pd.DataFrame()
submission['PassengerId'] = pass_id
submission['Survived'] = predictions
submission.to_csv('submission.csv', index=False)